In [3]:
import re
import string
import pandas as pd
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

from cleanlab import Datalab

In [5]:
# Read parquet data into pandas DataFrame
df = pd.read_parquet('/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/homework_3/pr4/synthetic_reviews.parquet')

raw_texts, labels = df["Email"].values, df["Category"].values
num_classes = len(set(labels))


In [6]:
transformer = SentenceTransformer('distiluse-base-multilingual-cased-v2')
text_embeddings = transformer.encode(raw_texts)

/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/env/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)
/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
model = LogisticRegression(max_iter=400)
pred_probs = cross_val_predict(model, text_embeddings, labels, method="predict_proba")

/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/env/lib/python3.9/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:1415: RuntimeWarning: Number of classes in training fold (8) does not match total number of classes (9). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(


In [ ]:
data_dict = {"texts": raw_texts, "labels": labels}
lab = Datalab(data_dict, label_name="labels")
lab.find_issues(pred_probs=pred_probs, features=text_embeddings)

In [14]:
lab.report()

Dataset Information: num_examples: 95, num_classes: 9

Here is a summary of various issues found in your data:

     issue_type  num_issues
 near_duplicate           6
          label           4
        outlier           1
        non_iid           1
class_imbalance           1

Learn about each issue: https://docs.cleanlab.ai/stable/cleanlab/datalab/guide/issue_type_description.html
See which examples in your dataset exhibit each issue via: `datalab.get_issues(<ISSUE_NAME>)`

Data indices corresponding to top examples of each issue are shown below.


------------------ near_duplicate issues -------------------

About this issue:
	A (near) duplicate issue refers to two or more examples in
    a dataset that are extremely similar to each other, relative
    to the rest of the dataset.  The examples flagged with this issue
    may be exactly duplicated, or lie atypically close together when
    represented as vectors (i.e. feature embeddings).
    

Number of examples with this issue: 6

In [27]:
label_issues = lab.get_issues("label")
identified_label_issues = label_issues[label_issues["is_label_issue"] == True]

,Email,Category
3,"Sehr geehrter Kundenservice, ich möchte mein I...",4
21,Ich habe den Service von Ihnen bereits gekündi...,4
47,"Guten Tag, können Sie mir bitte die Zahlungsei...",2
92,"Guten Tag, ich habe ein Problem mit der letzte...",2


In [31]:
outlier_issues = lab.get_issues("outlier")
label_issues[outlier_issues["is_outlier_issue"] == True]

,is_label_issue,label_score,given_label,predicted_label
16,False,0.14025,1,6


In [79]:
duplicate_issues = lab.get_issues("near_duplicate")
duplicate_issues_idx = duplicate_issues[duplicate_issues["is_near_duplicate_issue"] == True].index.to_numpy()
duplicate_issues_idx_2 = duplicate_issues[duplicate_issues["is_near_duplicate_issue"] == True].near_duplicate_sets.to_numpy()

duplicate_issues_idx_2 = [item for sublist in duplicate_issues_idx_2 for item in sublist]

duplicates_df = pd.concat([df.loc[duplicate_issues_idx].reset_index(drop=True), 
                           df.loc[duplicate_issues_idx_2].reset_index(drop=True)], axis=1)
duplicates_df.columns = ['Original_Email', 'Original_Category', 'Duplicate_Email', 'Duplicate_Category']
duplicates_df

,Original_Email,Original_Category,Duplicate_Email,Duplicate_Category
0,"Sehr geehrte Damen und Herren, ich möchte um d...",8,"Sehr geehrte Damen und Herren, ich möchte eine...",8
1,"Guten Tag, ich möchte meinen Vertrag schnellst...",4,"Guten Tag, ich möchte den Vertrag so schnell w...",4
2,"Guten Tag, ich möchte meine Bestellung stornie...",4,"Guten Tag, ich möchte meine Bestellung stornie...",4
3,"Sehr geehrte Damen und Herren, ich möchte eine...",8,"Sehr geehrte Damen und Herren, ich möchte um d...",8
4,"Guten Tag, ich möchte meine Bestellung stornie...",4,"Guten Tag, ich möchte meine Bestellung stornie...",4
5,"Guten Tag, ich möchte den Vertrag so schnell w...",4,"Guten Tag, ich möchte meinen Vertrag schnellst...",4


In [82]:
duplicates_df.iloc[0,0]

'Sehr geehrte Damen und Herren, ich möchte um die Kopie meines Vertrags bitten.'

In [77]:
df.iloc[79,0]

'Sehr geehrte Damen und Herren, ich möchte eine Kopie meines Vertrags anfordern.'

In [71]:
df.iloc[duplicate_issues_idx]

,Email,Category
1,"Sehr geehrte Damen und Herren, ich möchte um d...",8
60,"Guten Tag, ich möchte meinen Vertrag schnellst...",4
65,"Guten Tag, ich möchte meine Bestellung stornie...",4
79,"Sehr geehrte Damen und Herren, ich möchte eine...",8
88,"Guten Tag, ich möchte meine Bestellung stornie...",4
94,"Guten Tag, ich möchte den Vertrag so schnell w...",4


In [68]:
duplicate_issues_idx

array([ 1, 60, 65, 79, 88, 94])

In [1]:
import re
import string
import pandas as pd
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

from cleanlab import Datalab

import warnings
warnings.filterwarnings("ignore")

# Read parquet data into pandas DataFrame
df = pd.read_parquet('/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/homework_3/pr4/synthetic_reviews.parquet')

raw_texts, labels = df["Email"].values, df["Category"].values
num_classes = len(set(labels))


transformer = SentenceTransformer('distiluse-base-multilingual-cased-v2')
text_embeddings = transformer.encode(raw_texts)

model = LogisticRegression(max_iter=400)
pred_probs = cross_val_predict(model, text_embeddings, labels, method="predict_proba")

data_dict = {"texts": raw_texts, "labels": labels}
lab = Datalab(data_dict, label_name="labels",verbosity = 0)
lab.find_issues(pred_probs=pred_probs, features=text_embeddings)


/Users/daniil.yefimov/Desktop/Github/machine_learning_in_production/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process jus

In [89]:
label_issues = lab.get_issues("label")
label_issues_idx = label_issues[label_issues["is_label_issue"] == True].index.to_numpy()
print(df.iloc[label_issues_idx])

outlier_issues = lab.get_issues("outlier")
outlier_issues_idx = outlier_issues[outlier_issues["is_outlier_issue"] == True].index.to_numpy()
print(df.iloc[outlier_issues_idx])



duplicate_issues = lab.get_issues("near_duplicate")
duplicate_issues_idx = duplicate_issues[duplicate_issues["is_near_duplicate_issue"] == True].index.to_numpy()
duplicate_issues_idx_2 = duplicate_issues[duplicate_issues["is_near_duplicate_issue"] == True].near_duplicate_sets.to_numpy()

duplicate_issues_idx_2 = [item for sublist in duplicate_issues_idx_2 for item in sublist]

duplicates_df = pd.concat([df.loc[duplicate_issues_idx].reset_index(drop=True), 
                           df.loc[duplicate_issues_idx_2].reset_index(drop=True)], axis=1)
duplicates_df.columns = ['Original_Email', 'Original_Category', 'Duplicate_Email', 'Duplicate_Category']
print(duplicates_df)

                                                Email  Category
3   Sehr geehrter Kundenservice, ich möchte mein I...         4
21  Ich habe den Service von Ihnen bereits gekündi...         4
47  Guten Tag, können Sie mir bitte die Zahlungsei...         2
92  Guten Tag, ich habe ein Problem mit der letzte...         2
                                                Email  Category
16  Ich habe Fragen zu Ihrer Geschäftslösung und w...         1
                                      Original_Email  Original_Category  \
0  Sehr geehrte Damen und Herren, ich möchte um d...                  8   
1  Guten Tag, ich möchte meinen Vertrag schnellst...                  4   
2  Guten Tag, ich möchte meine Bestellung stornie...                  4   
3  Sehr geehrte Damen und Herren, ich möchte eine...                  8   
4  Guten Tag, ich möchte meine Bestellung stornie...                  4   
5  Guten Tag, ich möchte den Vertrag so schnell w...                  4   

                          